In [1]:
#install packages not native to Colabs
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 35.6 MB/s 
     |████████████████████████████████| 636 kB 38.7 MB/s 
     |████████████████████████████████| 3.3 MB 36.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [6]:
#%tensorflow_version 2.x
import tensorflow as tf
import transformers
import numpy as np
import pandas as pd
import sklearn
import csv
import nltk
import random
import torch
import re
random.seed(1234) 
np.random.seed(1234)
torch.manual_seed(1234) 
tf.random.set_seed(1234)
from transformers import InputExample, InputFeatures
from transformers import AdamW, get_linear_schedule_with_warmup
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams

from torch import nn, optim
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [7]:
totaldf=pd.read_csv('/content/drive/MyDrive/NLP Proj 2/Total_data.csv', header=0, encoding='latin-1')
totaldf.Label.replace(('netflix','spotify','other'),(1,0,2), inplace=True)
traindf, validdf = train_test_split(totaldf, test_size=0.2, shuffle=True, stratify=totaldf["Label"])

print(traindf)
print(validdf)

                                                  Text  Label
98                       whatever cheerful from neffex      0
287  The small white buoys marked the location of h...      2
422   Iron pyrite is the most foolish of all minerals.      2
517  It's much more difficult to play tennis with a...      2
204    Find me the film with the Disney villain Jaffar      1
..                                                 ...    ...
63            anything I could dance to or have a beat      0
149  any movies with henry cavill especially is he ...      1
506                              IÕm a living furnace.      2
447  She felt that chill that makes the hairs on th...      2
84                         A Taylor Swift song for now      0

[460 rows x 2 columns]
                                                  Text  Label
410  Flesh-colored yoga pants were far worse than e...      2
342          There's a message for you if you look up.      2
15                        play soundtrack from

In [8]:
print(traindf.groupby('Label').nunique())
print(validdf.groupby('Label').nunique())

       Text
Label      
0        90
1        90
2       243
       Text
Label      
0        23
1        23
2        66


In [9]:
#Function for encoding data
class TrainDataset(Dataset):

    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding=True,
        return_attention_mask=True,
        return_tensors='pt',
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'review_text': review,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [10]:
#Function for DataLoader
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = TrainDataset(
    reviews=df.Text.to_numpy(),
    targets=df.Label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [11]:
#Import tokenizer and set the Max Length after tokenization
from transformers import AlbertTokenizer
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v1')

MAX_LEN = 0

# For every sentence...
for sent in pd.concat([traindf['Text'],validdf['Text']],ignore_index=True):
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    # Update the maximum sentence length.
    MAX_LEN = max(MAX_LEN, len(input_ids))

print('Max sentence length: ', MAX_LEN)


Max sentence length:  37


In [12]:
#Process the data from train, validation, test
BATCH_SIZE = 2
train_data_loader = create_data_loader(traindf, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(validdf, tokenizer, MAX_LEN, BATCH_SIZE)
#test_data_loader = create_data_loader(testdf, tokenizer, MAX_LEN, BATCH_SIZE)

In [13]:
#import the model and save to GPU/CPU
from transformers import AlbertForSequenceClassification
model = AlbertForSequenceClassification.from_pretrained('albert-base-v1', num_labels = 3)
model = model.to(device)

#Can reduce Epochs if insufficient memory
EPOCHS = 3
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [14]:
from sklearn import metrics

#Create the training function and display the results
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d["input_ids"].reshape(2,MAX_LEN).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

In [15]:
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].reshape(2,MAX_LEN).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1

    return acc / counter, np.mean(losses)

In [16]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,     
        optimizer, 
        device, 
        scheduler, 
        len(traindf)
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader, 
        device, 
        len(validdf)
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        #torch.save(model.state_dict(), '/content/drive/MyDrive/Assignment 2/Models.py')
        best_accuracy = val_acc

Epoch 1/3
----------
Train loss 0.2688829749055794 Train accuracy 0.8934782608695652
Val loss 0.2853087870472368 Val accuracy 0.9568965517241379

Epoch 2/3
----------
Train loss 0.07929218718086106 Train accuracy 0.9847826086956522
Val loss 0.14683096473966323 Val accuracy 0.9741379310344828

Epoch 3/3
----------
Train loss 0.03421166550504732 Train accuracy 0.9934782608695653
Val loss 0.13411599233373198 Val accuracy 0.9827586206896551

CPU times: user 28.1 s, sys: 1.02 s, total: 29.2 s
Wall time: 29.5 s


In [17]:
#Save parameters for future retrieval
torch.save(model.state_dict(), '/content/drive/MyDrive/NLP Proj 2/AlbertModelparams.pt')

In [18]:
#Create function to formulate prediction on test set
def predict_sentiment(text):
    review_text = text

    encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    truncation=True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=False,
    return_attention_mask=True,
    return_tensors='pt',
    )

    input_ids = pad_sequences(encoded_review['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    input_ids = input_ids.astype(dtype = 'int64')
    input_ids = torch.tensor(input_ids) 

    attention_mask = pad_sequences(encoded_review['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    attention_mask = attention_mask.astype(dtype = 'int64')
    attention_mask = torch.tensor(attention_mask) 

    input_ids = input_ids.reshape(1,MAX_LEN).to(device)
    attention_mask = attention_mask.to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    outputs = outputs[0][0].cpu().detach()

    probs = F.softmax(outputs, dim=-1).cpu().detach().numpy().tolist()
    _, prediction = torch.max(outputs, dim =-1)

    #print("Positive score:", probs[1])
    #print("Negative score:", probs[0])
    #print(f'Review text: {review_text}')
    return class_names[prediction]

In [19]:
from transformers import AlbertForSequenceClassification
model = AlbertForSequenceClassification.from_pretrained('albert-base-v1', num_labels = 3)
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP Proj 2/AlbertModelparams.pt'))
model = model.to(device)
class_names=['spotify','netflix','other']

Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [22]:
test=" Play champion by Dante Bowe"
predict_sentiment(test)

'spotify'